In [8]:
from tvDatafeed import TvDatafeed, Interval
from datetime import datetime, timedelta
import pandas as pd

username = 'jdonadelli'
password = open('pass.txt', 'r').read()

tv = TvDatafeed(username, password)

ERROR:tvDatafeed.main:error while signin


In [2]:
# tv.search_symbol('PETR4','BMFBOVESPA')
# df = tv.get_hist(symbol='PETR4',exchange='BMFBOVESPA',interval=Interval.in_5_minute, n_bars= 5000)
# df.head(3)

In [3]:
url  = {'ibra50':'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/IBXLDia_15-12-23.csv',
        'ibra100':'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/IBXXDia_15-12-23.csv',
        'idiv':'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/IDIVDia_15-12-23.csv',
        'ibov':'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/IBOVDia_15-12-23.csv',
        'small':'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/SMLLDia_15-12-23.csv'
}

indice = pd.read_csv(url['small'] ,
                     sep=';' , decimal=',' , thousands='.' , header=1 , skiprows=0 , skipfooter=2 , encoding='latin-1',
                     index_col=False , engine='python')

indice.head()

,Código,Ação,Tipo,Qtde. Teórica,Part. (%)
0,RRRP3,3R PETROLEUM,ON NM,235661666,2.122
1,TTEN3,3TENTOS,ON NM,113777586,0.402
2,ABCB4,ABC BRASIL,PN N2,66485201,0.516
3,AERI3,AERIS,ON NM,690257429,0.214
4,AESB3,AES BRASIL,ON NM,317103937,1.253


In [4]:
for idx,row in indice.iterrows():
    i=row['Código']
    if idx == 0:       
        df =  tv.get_hist(symbol= i,exchange='BMFBOVESPA', n_bars= 800)
        df.rename(columns={ 'open':i+'_open' , 'high' : i+'_high' , 
                       'low' : i+'_low' , 'close' : i+'_close',
                       'volume' : i+'_volume'},inplace=True)
        del df['symbol']
    else:
        df1 =  tv.get_hist(symbol= i,exchange='BMFBOVESPA', n_bars= 800)
        df1.rename(columns={ 'open':i+'_open' , 'high' : i+'_high' , 
                       'low' : i+'_low' , 'close' : i+'_close',
                       'volume' : i+'_volume'},inplace=True)
        del df1['symbol']

        df = pd.merge(df,df1,how='inner',left_index=True,right_index=True)

df.to_csv("data/tvohlc.csv" , sep=';' , decimal='.')

In [ ]:
def import_data(csv): 
    """ Renomeando as colunas para aritmetica com dataframe """
    def extrair_caracteres(s):
        import re
        pattern = re.compile(r"([^_]+)_")
    
        match = pattern.match(s)
        if match:
                return match.group(1)
        else:
             return None
    
    df = pd.read_csv(csv, sep=';' , decimal='.')
    df.index = pd.to_datetime(df['datetime'])#, format = "%y-%m-%d")
    df = df.drop('datetime', axis=1)
    
    close_columns    = []
    high_columns     = []
    low_columns      = []
    open_columns     = []
    volume_columns   = []

    for i in df.columns:
        if "close" in i:
            close_columns.append(i)
        elif "high" in i:
            high_columns.append(i)
        elif "low" in i:
            low_columns.append(i)
        elif "open_int" in i:
            open_int_columns.append(i)
        elif "open" in i:
            open_columns.append(i)
        elif "volume" in i:
            volume_columns.append(i)

    dicH = {s:extrair_caracteres(s) for s in high_columns}
    dicL = {s:extrair_caracteres(s) for s in low_columns}
    dicO = {s:extrair_caracteres(s) for s in open_columns}
    dicC = {s:extrair_caracteres(s) for s in close_columns}
    dicV = {s:extrair_caracteres(s) for s in volume_columns}
    
    close_df    = df[close_columns].rename(columns=dicC)
    high_df     = df[high_columns].rename(columns=dicH)
    low_df      = df[low_columns].rename(columns=dicL)
    open_df     = df[open_columns].rename(columns=dicO)
    volume_df   = df[volume_columns].rename(columns=dicV)
 
    return [close_df, high_df, low_df, open_df, volume_df]